In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.opts.skotch import Skotch
from src.opts.askotch import ASkotch
from src.logger import Logger
from src.utils import set_random_seed, load_data

import torch
import wandb

In [3]:
dataset = 'homo'
task = 'regression'
kernel_params = {'type': 'l1_laplace', 'sigma': 5120}
lambd = 1e-3
opt = 'askotch'
b = 10
beta = 0
precond_params = {'type': 'nystrom', 'r': 100}
max_iter = 3000
log_freq = 10
seed = 0
device = 0
wandb_project = 'refactor_test'

In [4]:
set_random_seed(seed)

In [5]:
experiment_args = {
    'dataset': dataset,
    'task': task,
    'kernel_params': kernel_params,
    'lambd': lambd,
    'opt': opt,
    'b': b,
    'beta': beta,
    'precond_params': precond_params,
    'max_iter': max_iter,
    'log_freq': log_freq,
    'seed': seed,
    'device': f'cuda:{device}'
}

In [6]:
with wandb.init(project=wandb_project, config=experiment_args):
    # Access the experiment configuration
    config = wandb.config

    # Load the dataset
    Xtr, Xtst, ytr, ytst = load_data(config.dataset, config.seed, config.device)

    # Select the optimizer
    if config.opt == 'skotch':
        opt = Skotch(config.b, config.precond_params)
    elif config.opt == 'askotch':
        opt = ASkotch(config.b, config.beta, config.precond_params)
    else:
        raise ValueError("Only Skotch/ASkotch are supported")
    
    # Initialize at 0
    a0 = torch.zeros(Xtr.shape[0], device=config.device)

    # Initialize the logger
    logger = Logger(config.log_freq)
    
    # Run the optimizer
    with torch.no_grad():
        opt.run(Xtr, ytr, Xtst, ytst, config.kernel_params, config.lambd, config.task,
                a0, config.max_iter, config.device, logger)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pratikrathore8 (sketchy-opts). Use `wandb login --relogin` to force relogin


[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : 

Traceback (most recent call last):
  File "/tmp/ipykernel_1409876/2582466700.py", line 24, in <module>
    opt.run(Xtr, ytr, Xtst, ytst, config.kernel_params, config.lambd, config.task,
  File "/home/pratikr/fast_krr/src/opts/askotch.py", line 68, in run
    logger.compute_log_reset(
  File "/home/pratikr/fast_krr/src/logger.py", line 48, in compute_log_reset
    metrics_dict = self.compute_metrics(K_lin_op, K_tst, a, b, b_tst, b_org_norm,
  File "/home/pratikr/fast_krr/src/logger.py", line 23, in compute_metrics
    pred = K_tst @ a
  File "/home/pratikr/fast_krr/fast_krr_env/lib/python3.10/site-packages/pykeops/common/lazy_tensor.py", line 2524, in __matmul__
    Kv = Kv.sum(Kv.dim() - 2, **kwargs)  # Matrix-vector or Matrix-matrix product
  File "/home/pratikr/fast_krr/fast_krr_env/lib/python3.10/site-packages/pykeops/common/lazy_tensor.py", line 2096, in sum
    return self.reduction("Sum", axis=axis, **kwargs)
  File "/home/pratikr/fast_krr/fast_krr_env/lib/python3.10/site-package

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


iter_time,▃▄▄▆▂▅▂▁▅█▁▅▄▅▃▆▃▄▄▄▇▃▄▄▄▄▅▄▃▃▂▃▆▅▁▂▆▂▆▆
rel_residual,█▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
smape,█▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_mse,█▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,██▇▆▅▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
iter_time,0.47065
rel_residual,0.00449
smape,0.02643
test_mse,0.0253
train_loss,-1652867.375


KeyboardInterrupt: 